In [86]:
import openai
import os
import pandas as pd
import time
text_file = open("API_key.txt", "r")

folder_path = 'repo' # Replace with the path to your folder
file_paths_details = []

select_files = input('Update File List? : ')

def highlight(row):
    if row['Train_Flag'] > 0:
        return pd.Series(['background-color: yellow']*len(row), index=row.index)
    else:
        return pd.Series(['']*len(row), index=row.index)


if(select_files == 'y'):
    # Walk through all files and directories within the folder
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            # Append the path to each file to the file_paths list
            file_paths_details.append(os.path.join(root, filename))
    df4 = pd.DataFrame(file_paths_details)
    df4['Train_Flag'] = 0
    df4.columns = ["filepath","Train_Flag"]
    df4.to_csv("df4.csv", index=False)
    display(df4)
    print("Please Update the dataframe CSV File")
else:
    df4 = pd.read_csv('df4.csv')
    df4.columns = ["filepath","Train_Flag"]
    df5 = df4
    df5 = df5.style.apply(highlight, axis=1)
    display(df5)

Update File List? : n


,filepath,Train_Flag
0,repo/index.html,1
1,repo/.DS_Store,0
2,repo/login-page.js,1
3,repo/style.css,1


In [89]:
#read OPEN AI API KEY

text_file = open("API_key.txt", "r")
openai.api_key =  text_file.read()

text_file.close()

line_embeddings = []
    
def split_file_by_line(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        line_number = 0
        for i in lines:
            line_embeddings.append([filename,i,line_number])
            line_number +=1
    return 

comment_chars = {
    '.html':"<!-",
    '.css':"/*",
    '.js':"//",
    '.py':"#"
}

blocks = []
def split_file_by_character(filename,character):
    with open(filename, 'r') as f:
        lines = f.readlines()

    start_line = 0
    prev_end_line = -1
    for i, line in enumerate(lines):
        if character in line:
            prev_end_line = i
            extracted_lines = lines[start_line:prev_end_line]

            # join the extracted lines into a string
            extracted_text = "".join(extracted_lines)
            if extracted_text != "":
                blocks.append([filename,start_line,prev_end_line,extracted_text])
            #print(filename)
            start_line = i 
    
    extracted_lines = lines[prev_end_line+1:len(lines)]
    # join the extracted lines into a string
    extracted_text = "".join(extracted_lines)
    blocks.append([filename,prev_end_line,len(lines),extracted_text])
    
    return 


for i in range(0,len(df4)):
    if (df4.iloc[i][1]==1):
        print(df4.iloc[i][0])
        split_file_by_line(df4.iloc[i][0])
        file_extension = os.path.splitext(df4.iloc[i][0])[1]
        split_file_by_character(df4.iloc[i][0],comment_chars[file_extension])

print("Total number of functions extracted:", len(blocks))

repo/index.html
repo/login-page.js
repo/style.css
Total number of functions extracted: 10


In [90]:
df = pd.DataFrame(line_embeddings)
df2 = pd.DataFrame(blocks)
df.columns = ["filepath","Code","LineNumber"]
display(df)
df2.columns = ["filepath","BlockStart","BlockStop","Code"]
display(df2)

,filepath,Code,LineNumber
0,repo/index.html,<!DOCTYPE html>\n,0
1,repo/index.html,"<html lang=""en"">\n",1
2,repo/index.html,\n,2
3,repo/index.html,<!- File References ->\n,3
4,repo/index.html,\n,4
...,...,...,...
129,repo/style.css,font-weight: bold;\n,80
130,repo/style.css,background-color: #33c1ea;\n,81
131,repo/style.css,cursor: pointer;\n,82
132,repo/style.css,outline: none;\n,83


,filepath,BlockStart,BlockStop,Code
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde..."
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex..."
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log..."
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...


In [92]:
df['code_embedding'] = ''
def get_embedding(task):
    time.sleep(2.5)
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']
i=0
for ind in df.index:
        i+=1
        df['code_embedding'][ind] = get_embedding(df['Code'][ind])
        print(round(100*i/len(df) + "%")
print("Done")
df.to_csv("df.csv", index=False)


/var/folders/tb/txlhvbmn2fgct223r9bxzyc80000gn/T/ipykernel_70872/1648596614.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_embedding'][ind] = get_embedding(df['Code'][ind])


0.746268656716418
1.492537313432836
2.2388059701492535
2.985074626865672
3.7313432835820897
4.477611940298507
5.223880597014926
5.970149253731344
6.7164179104477615
7.462686567164179
8.208955223880597
8.955223880597014
9.701492537313433
10.447761194029852
11.194029850746269
11.940298507462687
12.686567164179104
13.432835820895523
14.17910447761194
14.925373134328359
15.671641791044776
16.417910447761194
17.16417910447761
17.91044776119403
18.65671641791045
19.402985074626866
20.149253731343283
20.895522388059703
21.64179104477612
22.388059701492537
23.134328358208954
23.880597014925375
24.62686567164179
25.37313432835821
26.119402985074625
26.865671641791046
27.611940298507463
28.35820895522388
29.104477611940297
29.850746268656717
30.597014925373134
31.34328358208955
32.08955223880597
32.83582089552239
33.582089552238806
34.32835820895522
35.07462686567164
35.82089552238806
36.56716417910448
37.3134328358209
38.059701492537314
38.80597014925373
39.55223880597015
40.298507462686565
41.

In [93]:
df = pd.read_csv('df.csv')
df['code_embedding'] = df.code_embedding.apply(lambda x: [float(y) for y in x[1:-1].split(",")])
display(df)

,filepath,Code,LineNumber,code_embedding
0,repo/index.html,<!DOCTYPE html>\n,0,"[-0.012485446408390999, 0.002962589031085372, ..."
1,repo/index.html,"<html lang=""en"">\n",1,"[-0.008912181481719017, 0.010274510830640793, ..."
2,repo/index.html,\n,2,"[0.002500855131074786, -0.018236210569739342, ..."
3,repo/index.html,<!- File References ->\n,3,"[-0.025017328560352325, -0.0020156055688858032..."
4,repo/index.html,\n,4,"[0.0025078936014324427, -0.018215738236904144,..."
...,...,...,...,...
129,repo/style.css,font-weight: bold;\n,80,"[-0.020041272044181824, 0.0022615562193095684,..."
130,repo/style.css,background-color: #33c1ea;\n,81,"[-0.008634612895548344, -0.0038089891895651817..."
131,repo/style.css,cursor: pointer;\n,82,"[-0.035627853125333786, -0.0004950165166519582..."
132,repo/style.css,outline: none;\n,83,"[-0.03516428545117378, 0.01233269739896059, 0...."


In [94]:
import numpy 
import matplotlib
task = 'Add height and width limits for image below login header'
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query):
    embedding = get_embedding(code_query)
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(100)
    return res

res = search_functions(df, task)

In [95]:
res = df.sort_values('similarities', ascending=False).head(round(0.05*len(df)))
display(res)
print(len(res))

,filepath,Code,LineNumber,code_embedding,similarities
103,repo/style.css,#login-form {\n,54,"[-0.003656495828181505, 0.014500855468213558, ...",0.790694
17,repo/index.html,"<h1 id=""login-header"">Login</h1>\n",17,"[0.0010761611629277468, 0.025847628712654114, ...",0.786548
8,repo/index.html,<title>Login</title>\n,8,"[-0.0014111034106463194, 0.011160314083099365,...",0.782597
14,repo/index.html,<!- Login Page Holder ->\n,14,"[-0.03041839227080345, 0.023289525881409645, -...",0.781226
102,repo/style.css,/* Login form feild styles */\n,53,"[-0.008210216648876667, 0.025797365233302116, ...",0.780047
123,repo/style.css,#login-form-submit {\n,74,"[-0.014604851603507996, 0.010971074923872948, ...",0.775628
23,repo/index.html,"<form id=""login-form"">\n",23,"[-0.018279781565070152, 0.011316388845443726, ...",0.774578


7


In [97]:
df3 = pd.DataFrame()

In [96]:
df2["Hits"] = 0
display(df2)

,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,0
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",0
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",0
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,0


In [98]:
# group by filepath and concatenate LineNumber
df3 = res.groupby("filepath").agg({"LineNumber": list}).reset_index()

display(df3)

,filepath,LineNumber
0,repo/index.html,"[17, 8, 14, 23]"
1,repo/style.css,"[54, 53, 74]"


In [99]:
import numpy as np
def count_lines(filepath, start, stop):
    count = 0
    for i in df3[df3['filepath']==filepath]['LineNumber']:
        #print(i)
        for j in i:
            if(j>=start and j<stop):
                count +=1
    return count

# apply the function to each row of df2 and create a new column
df2['Hits'] = df2.apply(lambda row: count_lines(row['filepath'], row['BlockStart'], row['BlockStop']), axis=1)
display(df2)
df2.to_csv("df2.csv", index=False)


,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,3


In [100]:
display(df2)
df2.to_csv("df2.csv", index=False)

,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,3


In [101]:
df2 = df2.sort_values('Hits', ascending=False)   
print(df2.iloc[0])

filepath                                         repo/style.css
BlockStart                                                   53
BlockStop                                                    85
Code          #login-form {\n  align-self: flex-start;\n  di...
Hits                                                          3
Name: 9, dtype: object
